In [46]:
import pandas as pd
data = pd.read_feather("dataFrame.feather")
print(data.shape)

(2225, 3)


In [ ]:
# that would be great but you dont have enough memory
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('/home/robertb/Desktop/text_embeddings/project/GoogleNews-vectors-negative300.bin', binary=True)  


In [92]:
# learn embedings
import gensim 
from nltk.tokenize import sent_tokenize, word_tokenize

text_new_line = [data.iloc[i,1].replace('\n', ". ") for i in range(data.shape[0])]
text_tokenized = [word_tokenize(text_new_line[i]) for i in range(len(text_new_line))]

model = gensim.models.Word2Vec(
        text_tokenized,
        size=100,
        window=6,
        min_count=1,
        workers=4,
        iter=10)


In [96]:
#TODO:
vocabulary = {word: vector.index for word, vector in embedding.vocab.items()} tk = Tokenizer(num_words=len(vocabulary)) tk.word_index = vocabulary tk.texts_to_sequences(samples)

w2v_model.wv.get_keras_embedding(train_embeddings=False)


[('6ft', 0.935685396194458),
 ('Crew', 0.9138823747634888),
 ('rumour', 0.9091072678565979),
 ('Lita', 0.9085713624954224),
 ('Hutus', 0.9054861664772034),
 ('Turner', 0.9048315286636353),
 ('Milton', 0.9020656943321228),
 ('Elia', 0.9010488986968994),
 ('Farina', 0.9005187749862671),
 ('ready-mixed', 0.8990009427070618)]

['Israel',
 'looks',
 'to',
 'US',
 'for',
 'bank',
 'chief',
 '.',
 '.',
 'Israel',
 'has',
 'asked',
 'a',
 'US',
 'banker',
 'and',
 'former',
 'International',
 'Monetary',
 'Fund',
 'director',
 'to',
 'run',
 'its',
 'central',
 'bank..',
 '.',
 'Stanley',
 'Fischer',
 ',',
 'vice',
 'chairman',
 'of',
 'banking',
 'giant',
 'Citigroup',
 ',',
 'has',
 'agreed',
 'to',
 'take',
 'the',
 'Bank',
 'of',
 'Israel',
 'job',
 'subject',
 'to',
 'approval',
 'from',
 'parliament',
 'and',
 'cabinet',
 '.',
 'His',
 'nomination',
 'by',
 'Prime',
 'Minister',
 'Ariel',
 'Sharon',
 'came',
 'as',
 'a',
 'surprise',
 ',',
 'and',
 'led',
 'to',
 'gains',
 'on',
 'the',
 'Tel',
 'Aviv',
 'stock',
 'market',
 '.',
 'Mr',
 'Fischer',
 ',',
 'who',
 'speaks',
 'fluent',
 'Hebrew',
 ',',
 'will',
 'have',
 'to',
 'become',
 'an',
 'Israeli',
 'citizen',
 'to',
 'take',
 'the',
 'job',
 '.',
 'The',
 'US',
 'says',
 'he',
 'will',
 'not',
 'have',
 'to',
 'give',
 'up',
 'US',
 'citizenship',
 

In [88]:
 [word_tokenize(a[i]) for i in range(len(text_tokenized))]

['Boeing unveils new 777 aircraft.',
 '.',
 "US aircraft firm Boeing has unveiled its new long-distance 777 plane, as it tries to regain its position as the industry's leading manufacturer.. .",
 'The 777-200LR will be capable of flying almost 11,000 miles non-stop, linking cities such as London and Sydney.',
 'Boeing, in contrast to European rival Airbus, hopes airlines will want to fly smaller aircraft over longer distances.',
 'Airbus, which overtook Boeing as the number one civilian planemaker in 2003, is focusing on so-called super jumbos.. .',
 'Analysts are divided over which approach is best and say that this latest tussle between Boeing and Airbus may prove to be a defining moment for the airline industry.',
 "Boeing plans to offer twin-engine planes that are able to fly direct to many of the world's airports, getting rid of the need for connecting flights.. .",
 'It is banking on smaller, slimmer planes such as the 777-200LR and its much-anticipated 787 Dreamliner plane, whic

In [70]:
data.shape[0]

2225